In [1]:
import re           
import os
import html

   #коллекция регулярных выражений
    
reg_categorie = re.compile('leaf.*?/categories/.*?.html', flags= re.DOTALL)
reg_page = re.compile('last last.*?page=\w\w?', flags= re.DOTALL)
reg_page_number = re.compile('last last.*?page=', flags= re.DOTALL)
reg_all_articles_on_page = re.compile ('<div id="main-wrap.*?class="pager_wrap">', flags=re.DOTALL)
reg_article = re.compile ('href="/articles/.*?[\."]', flags=re.DOTALL)
reg_text = re.compile ('<div class="content">.*?<div class="block block-vkaz" id="block-vkaz-vkaz_share42">', flags = re.DOTALL)
reg_auth = re.compile ('<div class="author heading--meta">.*?</div>', flags = re.DOTALL)         #регулярки для мета данных
reg_header = re.compile ('<h1 class="title title-story">.*?</h1>', flags = re.DOTALL)
reg_date = re.compile ('<div class="submitted heading--meta">\d\d\.\d\d\.\d\d', flags = re.DOTALL)
reg_cat = re.compile ('<a href=".*?" rel="tag" title="">.*?</a>')

main_directory = os.getcwd()            #директория, в которой лежит программа
mystem_directory = r"C:\mystem.exe "    #директория, в которой лежит mystem + пробел (он нужен в запросе к майстему)

In [2]:
def open_url (pageUrl):              #открывает страницу по заданному url
    import urllib.request  
    req = urllib.request.Request(pageUrl)
    with urllib.request.urlopen(req) as response:
        html = response.read().decode('utf-8')
    return html

In [3]:
def find_obj (html, regex):      # ищет url по заданному регулярному выражению
    found = regex.findall(html)
    return found

In [4]:
def clean_tag (regex, obj_to_clean):         #очищает найденные строки от ненужного мусора, заданного рег.
    obj_to_add = re.sub (regex,'', obj_to_clean)
    return obj_to_add

In [5]:
def real_urls (objects_to_add, regex):    #склеивает "чистый" юрл и имя сайта, выдаеn список юрл-ов
    objects_to_url =[]        #что надо убрать из объекта - в регэкс
    real_urls =[]
    for obj in objects_to_add:
        objects_to_url.append(clean_tag(regex, obj))
    for obj_to_url in objects_to_url:
        real_urls.append(default_Url+obj_to_url)
    return real_urls

In [6]:
def count_pages (cat_url):                   #считает кол-во страниц в разделе
    html = open_url (cat_url)
    number_of_pages_is_here = find_obj (html, reg_page)
    number_of_pages = clean_tag (reg_page_number, number_of_pages_is_here[0])
    return int(number_of_pages)

In [7]:
def articles (cat_url):                              #собирает массив с юрлами статей
    html = open_url (cat_url)
    code_where_articles_urls_are = find_obj (html, reg_all_articles_on_page)
    articles = find_obj (code_where_articles_urls_are[0], reg_article)
    articles_urls = list(set(real_urls (articles, 'href="/')))
    new_articles= []
    for article in articles_urls:
        new_articles.append(article.strip('\."')+'.html')
    return new_articles

In [8]:
def article_content (article_url):      #достает и очищает содержимое страницы со статьей
    text_code = open_url (article_url)
    text_is_here = find_obj (text_code, reg_text)
    text = clean_tag ('<.*?>', text_is_here[0])
    text = html.unescape(text)
    return text

In [9]:
def make_directory (newspaper, folder, year, month):                   #создает директорию
    path = main_directory
    #r'C:\Users\Maria\programming_2'
    directory = os.path.join (path, newspaper, folder, year, month)
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

In [10]:
def get_meta_data (page_url):                                 #получает мета-данные из статьи, возвращает словарь с ними
    html = open_url (page_url)
    metadata = {'path':'', 'author':'','sex':'','birthday':'','header':'', 'created':'', 'sphere':'публицистика', 'genre_fi':'','type':'','topic':'','chronotop ':'','style':'нейтральный',
'audience_age' :'н-возраст', 'audience_level': 'н-уровень','audience_size' :'городская','source' :'','publication':'Вечерняя Казань','publisher' :'',
'publ_year' :'','medium' : 'газета','country' : 'Россия','region' : 'Республика Татарстан','language': 'ru' }
    #path =  путь к неразмеченному файлу, достается в главной функции
    find_author = find_obj (html, reg_auth)
    metadata ['author'] = re.sub('<.*?>', '',find_author [0])
    find_header = find_obj (html, reg_header)
    metadata ['header'] = re.sub('<.*?>', '',find_header [0])
    find_date = find_obj (html, reg_date)
    metadata ['created'] = re.sub('<.*?>', '',find_date [0])
    find_categorie = find_obj (html, reg_cat)
    metadata ['topic'] = re.sub('<.*?>', '',find_categorie [0])
    metadata ['source'] = page_url
    metadata ['publ_year'] = '20'+re.sub('<.*?>\d\d\.\d\d\.', '',find_date [0])
    return metadata


In [11]:
def create_directory_and_go(metadata):                             #создает конкретную директорию и переходит в неё              
    make_directory ('newspaper','plain','20'+metadata['created'][6:8] , metadata['created'][3:5])
    os.chdir (os.path.join(main_directory,'newspaper','plain','20'+metadata['created'][6:8] , metadata['created'][3:5]))
    return os.path.join(main_directory,'newspaper','plain','20'+metadata['created'][6:8] , metadata['created'][3:5])

In [12]:
def article_into_file (i, metadata, article):                      #записывает содержимое статьи в файл
    with open(str(i)+'.txt', 'w', encoding='utf-8') as g:
        article_txt = '@au '+metadata['author']+'\n'+'@ti '+metadata['header']+'\n'+'@da '+metadata['created']+'\n'+'@topic '+metadata['topic']+'\n'+'@url '+metadata['source']+'\n'
        g.write(article_txt+article_content(article))

In [13]:
import csv                                                        #функции для создания csv с метаданными

def create_meta_csv (metadata):                                    #создаётся таблица
    with open('metadata.csv', "w", newline="") as file:
        columns = metadata.keys()
        writer = csv.DictWriter(file, delimiter = '\t', fieldnames=columns)
        writer.writeheader()
    
def add_data_to_csv (metadata):                                #добавляются данные конкретной статьи в таблицу
    with open('metadata.csv', "a", newline="") as file:
        columns = metadata.keys()
        writer = csv.DictWriter(file, delimiter = '\t', fieldnames=columns)
        writer.writerow(metadata)


In [14]:
def mystem_processing(folder_name, metadata, format_type, mode, i):              # обработка текста майстемом 
    outputdir = make_directory ('newspaper', folder_name,'20'+metadata['created'][6:8] , metadata['created'][3:5])
    metadata ['path'] = create_directory_and_go(metadata)+ os.sep +str(i)+'.txt'
    fl_dir = metadata['path']+' '
    os.system(mystem_directory + fl_dir + outputdir + os.sep + str(i) + ' '+ format_type +" "+ mode)

In [15]:
import os
import urllib

def main_func (default_Url):            #основная функция
    html = open_url (default_Url)
    categories = find_obj (html, reg_categorie)
    categories_urls = real_urls (categories, 'leaf.*?/') 
    i = 0
    for cat_url in categories_urls:
        pages = count_pages (cat_url)
        for page in range(pages):
            page_url = cat_url+'?page='+str(page)
            articles_urls = articles (page_url)
            try:
                for article in articles_urls:
                    print (article)
                    metadata = get_meta_data (article)
                    metadata ['path'] = create_directory_and_go(metadata)+str(i)+'.txt'
                    article_into_file(i, metadata, article)
                    os.chdir (os.path.join(main_directory,'newspaper'))
                    if i == 0:
                        create_meta_csv (metadata)
                    else:
                        pass
                    add_data_to_csv(metadata)
                    mystem_processing ('mystem-xml', metadata, '--format xml', '-di', i)
                    mystem_processing ('mystem-plain', metadata, '--format text', '-di', i)
                    i+=1
            except urllib.error.HTTPError: 
                pass
            except UnicodeEncodeError:
                pass
                
            
            

In [16]:
default_Url = 'http://www.evening-kazan.ru/'
main_func (default_Url)

http://www.evening-kazan.ru/articles/ne-prosto-moloko-vlasti-kazani-ozabotilis-sozdaniem-molochnogo-brenda.html
http://www.evening-kazan.ru/articles/v-kazani-ne-budut-perekryvat-mosty-radi-aviagonok-chtoby-ne-besit-avtomobilistov.html
http://www.evening-kazan.ru/articles/zapah-fekaliy-ne-daet-kazani-stat-samym-komfortnym-gorodom.html
http://www.evening-kazan.ru/articles/ella-pamfilova-ne-prostila-ekzamu-gubaydullinu-skandal-s-dosrochnym-golosovaniem.html
http://www.evening-kazan.ru/articles/ne-hodyat-tak-pust-hotya-by-platyat-tatarstan-ostanetsya-edinstvennym-regionom-v-pfo-gde-s-roditeley-detsadovcev-berut.html
http://www.evening-kazan.ru/articles/pomoshchnik-prezidenta-tatarstana-svistala-vseh-medikov-naverh.html
http://www.evening-kazan.ru/articles/dogovor-tatarstana-s-federalnym-centrom-prodlit-nelzya-zabyt.html
http://www.evening-kazan.ru/articles/v-park-menedzhery-kazani-vozmut-teh-kto-ponimaet.html
http://www.evening-kazan.ru/articles/tri-ministerstva-i-odin-goskomitet-tatarstan

http://www.evening-kazan.ru/articles/glava-rayona-v-tatarstane-na-predvybornoy-vstreche-s-narodom-posovetoval-skinutsya-na-detsad-i-dorogu-do-kladbishcha.html
http://www.evening-kazan.ru/articles/v-tatarstane-krepnet-nagradnoy-patriotizm.html
http://www.evening-kazan.ru/articles/krematoriya-v-kazani-ne-budet-eshche-30-let.html
http://www.evening-kazan.ru/articles/novogo-predsedatelya-molodezhnogo-komiteta-iskali-po-vsey-kazani-a-nashli-pod-bokom-u-mera.html
http://www.evening-kazan.ru/articles/delo-v-zapyatoy-oshibka-kazanskogo-chinovnika-edva-ne-stoila-9-gektarov-lesa.html
http://www.evening-kazan.ru/articles/vsyo-dlya-turistov-v-tatarstane-obnaruzhili-more-i-otlichnoe-zdravoohranenie.html
http://www.evening-kazan.ru/articles/tatarstanskie-kandidaty-v-deputaty-gosdumy-otkryli-koshelki.html
http://www.evening-kazan.ru/articles/vlasti-tatarstana-obeshchannyh-blag-do-pensii-zhdut.html
http://www.evening-kazan.ru/articles/rokirovka-na-bukvu-m-metshin-mozhet-zamenit-mutko.html
http://www.e

http://www.evening-kazan.ru/articles/deputat-vzbuntovalsya-protiv-goszhilfonda-tatarstana.html
http://www.evening-kazan.ru/articles/lider-edinorossov-tatarstana-salahutdinov-ne-pomogaet-sledstviyu-nayti-napavshih-na-nego.html
http://www.evening-kazan.ru/articles/ay-mgery-znat-oni-silny-vlasti-tatarstana-prosyat-zashchity-u-starshih-tovarishchey.html
http://www.evening-kazan.ru/articles/minnihanov-nemnogo-nedotyanul-do-rekorda-shaymieva.html
http://www.evening-kazan.ru/articles/rustam-minnihanov-voshel-vkontakt-s-narodom.html
http://www.evening-kazan.ru/articles/v-kazanskom-kremle-znayut-vsyo-o-svyazyah-almetevskogo-nachalstva-ili-pochyom-rodnaya-zemlya.html
http://www.evening-kazan.ru/articles/tatarstanskim-chinovnikam-ranshe-federalnyh-zapretili-obshchatsya-v-whatsapp.html
http://www.evening-kazan.ru/articles/v-tatarstane-hotyat-ukrepit-semi-ozhidaniem-razvoda.html
http://www.evening-kazan.ru/articles/rustam-minnihanov-ne-dal-trevozhnomu-segodnya-zaslonit-radostnoe-zavtra.html
http://

http://www.evening-kazan.ru/articles/poltora-navalnogo-eto-ne-dlya-nas.html
http://www.evening-kazan.ru/articles/minnihanovu-auknulos-dosrochnoe-golosovanie.html
http://www.evening-kazan.ru/articles/zavtra-v-pravitelstve-tatarstana-zapolnyat-rasstrelnuyu-vakansiyu.html
http://www.evening-kazan.ru/articles/inostrannyy-agent-protiv-gosudarstvennogo-tormoza.html
http://www.evening-kazan.ru/articles/k-2020-godu-kazancev-zadushat-nalogami.html
http://www.evening-kazan.ru/articles/o-chem-ne-govorit-prezident.html
http://www.evening-kazan.ru/articles/mnogovato-deputatov.html
http://www.evening-kazan.ru/articles/samye-deshevye-vybory.html
http://www.evening-kazan.ru/articles/vseh-vyberut-do-vyborov.html
http://www.evening-kazan.ru/articles/bogatye-v-tatarstane-za-partiyu-vlasti.html
http://www.evening-kazan.ru/articles/u-kommunistov-rossii-nashli-netradicionnye-znaki.html
http://www.evening-kazan.ru/articles/kto-posledniy-za-byulletenem.html
http://www.evening-kazan.ru/articles/halikov-otkazal

http://www.evening-kazan.ru/articles/zakrytyy-prokat.html
http://www.evening-kazan.ru/articles/gromkoe-molchanie.html
http://www.evening-kazan.ru/articles/rodina-vse-ne-nachinaetsya.html
http://www.evening-kazan.ru/articles/pervomay-i-dengi-v-kazani-za-solidarnost-doplachivayut.html
http://www.evening-kazan.ru/articles/agent-na-razoblachiteley-podast-v-sud.html
http://www.evening-kazan.ru/articles/eks-imama-kul-sharif-podderzhali-v-gossovete-rt.html
http://www.evening-kazan.ru/articles/vesna-glavy-poleteli.html
http://www.evening-kazan.ru/articles/za-husnullinym-prishel-doctor.html
http://www.evening-kazan.ru/articles/minnihanov-uvidel-trehmillionnuyu-kazan.html
http://www.evening-kazan.ru/articles/kazancy-prokontroliruyut-vybory-mera-moskvy.html
http://www.evening-kazan.ru/articles/muslyumovskiy-rayon-doveryat-zamglavy-kazani.html
http://www.evening-kazan.ru/articles/voskresnyy-gey-piket-v-kazani-zapretili.html
http://www.evening-kazan.ru/articles/bumazhnyh-barsov-pronumeruyut-i-budut

...

http://www.evening-kazan.ru/articles/deti-otdyhayut-na-chernom-more-s-gradusnikom-pod-myshkoy.html
http://www.evening-kazan.ru/articles/pacient-bugulminskoy-crb-umer-potomu-chto-vrachi-upustili-vremya.html
http://www.evening-kazan.ru/articles/na-nedonoshennyh-detey-sobirayut-dose.html
http://www.evening-kazan.ru/articles/krov-iz-nosu-tak-nuzhna-krov.html
http://www.evening-kazan.ru/articles/modernizaciya-zdravoohraneniya-nashlas-na-sklade.html
http://www.evening-kazan.ru/articles/baykeram-krasnogo-kresta-dali-zelenyy-svet.html
http://www.evening-kazan.ru/articles/net-vracha-net-zabolevaemosti-ili-kak-v-almetevske-infekcionnuyu-sluzhbu-uplotnili.html
http://www.evening-kazan.ru/articles/bastovat-meshaet-klyatva-gippokrata.html
http://www.evening-kazan.ru/articles/ros-rebenok-ros-i-vyros-iz-invalidnosti.html
http://www.evening-kazan.ru/articles/sayt-u-bolnicy-budet-a-vrachi.html
http://www.evening-kazan.ru/articles/sotrudniki-drkb-nachali-tushit-ogon-do-priezda-pozharnyh.html
http://www.

http://www.evening-kazan.ru/articles/vmesto-klyatvy-deklaraciya.html
http://www.evening-kazan.ru/articles/nedobryy-aybolit-za-simulyantku-otvetil.html
http://www.evening-kazan.ru/articles/pervymi-donorami-serdca-i-pecheni-v-kazani-stali-sobaki-i-koshki.html
http://www.evening-kazan.ru/articles/vtoroy-volny-grippa-ne-budet.html
http://www.evening-kazan.ru/articles/minzdrav-preduprezhdaet-statistika-uzhasaet.html
http://www.evening-kazan.ru/articles/vracham-v-ocherednoy-raz-poobeshchali-pribavku.html
http://www.evening-kazan.ru/articles/ya-ne-znala-chto-popav-v-bolnicu-nado-zhdat-smerti-rebenka.html
http://www.evening-kazan.ru/articles/diabetiki-napomnili-glave-minzdrava-o-principe-ne-navredi.html
http://www.evening-kazan.ru/articles/a-v-travmpunkte-kanikuly.html
http://www.evening-kazan.ru/articles/zdes-konchaetsya-vrachebnaya-tayna.html
http://www.evening-kazan.ru/articles/dlya-obitateley-mnogoetazhki-na-chuykova-kapremont-zakonchilsya-betonom-v-rozetkah-i-udarom-molnii.html
http://www

http://www.evening-kazan.ru/articles/kazancev-oblozhili-antennym-obrokom.html
http://www.evening-kazan.ru/articles/kto-ne-uspel-postavit-gazovyy-schetchik-tot-sekonomil.html
http://www.evening-kazan.ru/articles/kazancev-ne-zamanish-v-dvorniki-ni-besplatnym-zhilem-ni-pamyatnikami.html
http://www.evening-kazan.ru/articles/kazanskie-zhilishchniki-obyavili-nagradu-za-golovy-pohititeley-energosberegayushchego-oborudovaniya.html
http://www.evening-kazan.ru/articles/sotni-zhiteley-azino-1-otkazyvayutsya-oplachivat-poveshennye-na-nih-dolgi-za-otoplenie.html
http://www.evening-kazan.ru/articles/v-tatarstane-iz-avariynogo-zhilya-lyudey-pereselyayut-na-ulicu.html
http://www.evening-kazan.ru/articles/kommunalshchiki-v-tatarstane-vozmut-svoe-iz-zarplaty-dolzhnikov.html
http://www.evening-kazan.ru/articles/platit-za-proezd-v-kazani-stanet-dorozhe-i-neudobnee.html
http://www.evening-kazan.ru/articles/bani-bez-karaoke-i-striptiza-v-kazani-ischezayut-kak-vid.html
http://www.evening-kazan.ru/articles/gl

http://www.evening-kazan.ru/articles/zhiteli-novoy-sosnovki-golodayut-i-mitinguyut.html
http://www.evening-kazan.ru/articles/klyuch-bez-prava-peredachi-ili-igra-na-dengi-pod-senyu-socipoteki-2.html
http://www.evening-kazan.ru/articles/otdel-temnyh-del.html
http://www.evening-kazan.ru/articles/stan-dolgozhitelem-chtoby-dozhdatsya-kapremonta.html
http://www.evening-kazan.ru/articles/ostalos-nayti-alternativu-vizualnomu-tehobsluzhivaniyu.html
http://www.evening-kazan.ru/articles/kazancev-obschitali-za-vybory.html
http://www.evening-kazan.ru/articles/v-kazani-obyavlen-boykot-novoy-plate-za-detsad.html
http://www.evening-kazan.ru/articles/kto-vyvedet-vshey-esli-dezinfekcionnuyu-stanciyu-vyvedut.html
http://www.evening-kazan.ru/articles/kadastrovaya-palata-budet-konsultirovat-i-nochyu.html
http://www.evening-kazan.ru/articles/afgancy-vpered-za-lgotnuyu-pensiyu.html
http://www.evening-kazan.ru/articles/dumay-chto-podpisyvaesh.html
http://www.evening-kazan.ru/articles/a-nam-govoryat-chto-v-zhk

KeyboardInterrupt: 